# Generate Embeddings from Meal Description

In [ ]:
import boto3, os, json
from dotenv import load_dotenv

bedrock = boto3.client(
    "bedrock-runtime",
    region_name=
)

def get_embedding(text):
    payload = {"inputText": text}
    response = bedrock.invoke_model(
        modelId="amazon.titan-embed-text-v2:0",
        body=json.dumps(payload),
        contentType="application/json"
    )
    result = json.loads(response['body'].read())
    return result["embedding"]

# Example
if __name__ == "__main__":
    print(get_embedding("Grilled chicken salad with avocado and olive oil dressing"))


## Store in Memory

In [ ]:
from generate_embeddings import get_embedding

meal_store = {}

def store_meal(id, description):
    vector = get_embedding(description)
    meal_store[id] = {
        "description": description,
        "embedding": vector
    }

# Example
if __name__ == "__main__":
    store_meal("meal1", "Quinoa salad with chickpeas and lemon tahini dressing")
    print(meal_store)

## Store in Amazon S3

In [ ]:
import boto3, json, os
from dotenv import load_dotenv
from generate_embeddings import get_embedding

load_dotenv()

s3 = boto3.client(
    "s3",
    region_name=os.getenv("AWS_DEFAULT_REGION"),
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
)

BUCKET = "your-meal-embedding-bucket"

def store_meal_s3(id, description):
    embedding = get_embedding(description)
    body = json.dumps({"description": description, "embedding": embedding})
    s3.put_object(Bucket=BUCKET, Key=f"meals/{id}.json", Body=body)

# Example
if __name__ == "__main__":
    store_meal_s3("meal2", "Grilled salmon with steamed broccoli and almonds")


## Store in FAISS and Search by Nutritional Similarity

In [ ]:
import faiss
import numpy as np
from generate_embeddings import get_embedding

index = faiss.IndexFlatL2(1536)  # Titan V2 = 1536 dims
meal_ids = {}

def add_meal(id, description):
    vector = np.array([get_embedding(description)], dtype="float32")
    index.add(vector)
    meal_ids[index.ntotal - 1] = (id, description)

def search_similar_meals(query, top_k=3):
    q_vector = np.array([get_embedding(query)], dtype="float32")
    D, I = index.search(q_vector, top_k)
    return [meal_ids.get(i) for i in I[0]]

# Example
if __name__ == "__main__":
    add_meal("meal1", "Greek yogurt bowl with walnuts and honey")
    add_meal("meal2", "Boiled eggs with spinach and mushrooms")
    add_meal("meal3", "Avocado toast on rye bread with poached egg")

    print(search_similar_meals("High-protein vegetarian breakfast"))


## Store in OpenSearch Serverless (Scalable, Searchable)

In [ ]:
def store_in_opensearch(id, text):
    vec = get_embedding(text)
    opensearch.index(index="text-embeddings", id=id, body={"text": text, "embedding": vec})

def search_opensearch(query, k=3):
    vec = get_embedding(query)
    query_body = {
        "size": k,
        "knn": {
            "embedding": {
                "vector": vec,
                "k": k
            }
        }
    }
    response = opensearch.search(index="text-embeddings", body=query_body)
    return [(hit["_id"], hit["_source"]["text"]) for hit in response["hits"]["hits"]]

## Store in PostgreSQL DB with pgvector Vector Extension

In [ ]:
import psycopg2

conn = psycopg2.connect(
    host="your-db-endpoint",
    user="postgres",
    password="yourpass",
    dbname="postgres"
)

cur = conn.cursor()

text = "Lentils are rich in protein and ideal for plant-based diets."
embedding = get_embedding(text)

cur.execute("""
    INSERT INTO nutrition_embeddings (id, content, embedding)
    VALUES (%s, %s, %s)
    ON CONFLICT (id) DO NOTHING
""", ("lentils-001", text, embedding))
conn.commit()

# Search
search_vector = get_embedding("What are good vegan proteins?")
cur.execute("""
    SELECT content FROM nutrition_embeddings
    ORDER BY embedding <-> %s
    LIMIT 1
""", (search_vector,))
print("Match:", cur.fetchone()[0])


## Let’s walk through a sample:

In [ ]:
store_in_memory("m1", "Greek yogurt with chia seeds and honey")
store_in_s3("m2", "Boiled eggs with spinach and mushrooms")
store_in_faiss("m3", "Avocado toast with poached egg")
store_in_opensearch("m4", "Protein shake with almond butter and banana")

print(search_faiss("High-protein vegetarian breakfast"))
print(search_opensearch("High-protein vegetarian breakfast"))
